In [2]:
import kagglehub
import os
import tensorflow as tf 
from tensorflow.keras import layers, models
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tensorflow/python/keras/engine/training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [15]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print("Path to dataset: ", path)

train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

print("Train directory: ", train_dir)
print("Validation directory: ", val_dir)
print("Test directory: ", test_dir)

100%|██████████| 2.29G/2.29G [02:08<00:00, 19.2MB/s]

Extracting files...


Path to dataset:  /Users/koshiqhossain/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2
Train directory:  /kaggle/input/chest-xray-pneumonia/chest_xray/train
Validation directory:  /kaggle/input/chest-xray-pneumonia/chest_xray/val
Test directory:  /kaggle/input/chest-xray-pneumonia/chest_xray/test


In [19]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    color_mode="rgb",
    label_mode = "int"
)

val = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32,
    color_mode="rgb",
    label_mode = "int"
)

test = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    color_mode="rgb",
    label_mode = "int"
)

NotFoundError: Could not find directory /kaggle/input/chest-xray-pneumonia/chest_xray/train

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train = train.map(lambda x, y: (normalization_layer(x), y))
test = test.map(lambda x, y: (normalization_layer(x), y))
val = val.map(lambda x, y: (normalization_layer(x), y))

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

train = train.map(lambda x, y: (data_augmentation(x), y))

model = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1)
])

train = train.map(lambda x, y: (data_aug(x, training = True), y))

In [ ]:
base_model = tf.keras.applications.EfficientNetB4(
    include_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet'
)

In [ ]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

model.summary()

In [ ]:
history = model.fit(
    train,
    validation_data = val,
    epochs = 10,
    callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3, monitor = "val_loss")]
)
